# Fully-implicit compositional simulator

### Main assumptions
* Constant K-values for an arbitrary number of components
* Isothermal assumptions
* Constructing the Jacobian matrix with numerical differentiation
* Using block-wise storage of primary variables and residuals
* Using the original mass conservation equations instead of fractional flow curves
* Only for a 1-D domain

### Import necessary packages

In [ ]:
import numpy as np
import time
from numba import jit
import matplotlib.pyplot as plt

### Fluid properties

In [ ]:
@jit(nopython=True)
def RachfordRice(K_values, zc, flash_calcs_eps):
    a = 1 / (1 - np.max(K_values)) + flash_calcs_eps
    b = 1 / (1 - np.min(K_values)) - flash_calcs_eps

    max_iter = 200  # use enough iterations for V to converge
    for i in range(1, max_iter):
        V = 0.5 * (a + b)
        r = np.sum(zc * (K_values - 1) / (V * (K_values - 1) + 1))
        if abs(r) < 1e-12:
            break

        if r > 0:
            a = V
        else:
            b = V

    if i >= max_iter:
        print("Flash warning!!!")

    x = zc / (V * (K_values - 1) + 1)
    y = K_values * x

    return [V, 1 - V], [y, x]

def compute_saturation(ph, nu, rho):
    sat = np.zeros(max_num_phases)
    # Get phase saturations [volume fractions]
    Vtot = 0
    for j in ph:
        Vtot += nu[j] / rho[j]

    for j in ph:
        sat[j] = (nu[j] / rho[j]) / Vtot

    return sat

def calc_density(pressure):
    rho_gas = rho_ref_gas * (1 + compr_gas * (pressure - p_ref))
    rho_liquid = rho_ref_liquid * (1 + compr_liquid * (pressure - p_ref))

    return [rho_gas, rho_liquid]

def compute_props(p, zc):
    
    nu, x = RachfordRice(K_values, zc, 1e-12)

    x = np.array(x).reshape(max_num_phases, nc)
    nu = np.array(nu)

    ph = []
    for j in range(max_num_phases):
        if nu[j] > 0:
            ph.append(j)

    if len(ph) == 1:
        x = np.zeros((max_num_phases, nc))
        x[ph[0]] = zc

    for j in ph:
        rho = calc_density(p)

    s = compute_saturation(ph, nu, rho)

    return x, rho, s 

### Compute residual equations (mass conservation equations of components)

The conservation equation for each component has the following form:
\begin{equation}
g_{c,i}= \frac{V}{\Delta t}\left[\left(\phi\rho_T z_c\right)_i^{n+1} -\left(\phi\rho_T z_c\right)_i^{n}\right]
-T_{c,i+1/2} (p_{i+1}-p_{i})+T_{c,i-1/2} (p_i-p_{i-1}), \quad c = 1, \ldots,C.
\end{equation}

Here the transmissibility of the component can be defined as
\begin{equation}
T_{c,i+1/2}=\left(\frac{k_xA}{\Delta x}\right)_{i+1/2}\left(\sum_j x_{cj}\rho_j\lambda_j \right)_{i+1/2}=\Gamma_{i+1/2} \beta_{c,i+1/2}.
\end{equation}

The fluid part of transmissibility can be represented as 
\begin{equation}
\beta_{c,i+1/2} = \sum_j (x_{cj} \rho_j \lambda_j)_{i+1/2} = \sum_j \beta_{cj,i+1/2},
\end{equation}
\begin{equation}
\beta_{cj,i+1/2} = 
\begin{cases}
\beta_{cj,i}, \quad p_i > p_{i+1}, \\
\beta_{cj,i+1}, \quad p_i < p_{i+1}. \end{cases} 
\end{equation}
Note: due to gravity or capillarity, it is possible that the same component is moving in different directions within different phases.

For conventional compositional problems, it is easy to define new functions as a product of different nonlinear properties
\begin{equation}
\alpha_{c,i} = \left([1+c_r(p-p^0)]\rho_t z_c\right)_i, \quad \beta_{c,i} = \left(\rho_o x_c \frac{k_{ro}}{\mu_o} + \rho_g y_c \frac{k_{rg}}{\mu_g}\right)_i.
\label{comp_operators}
\end{equation}
Here we assume that $\phi = \phi^0(1+c_r(p-p^0)$, where $\phi^0$ is porosity at $p^0$ and $c_r$ is rock compressibility.

Notice that in this case, we use the general compositional formulation
\begin{equation}
\label{generic_formulation}
g_{c,i}= \frac{V\phi^0}{\Delta t}\left(\alpha_i^{n+1} -\alpha_i^{n}\right) %+ \Gamma^w \beta_c^w (p_i-p^w)
-(\Gamma \beta_c)_{i+1/2} (p_{i+1}-p_{i})+(\Gamma \beta_c)_{i-1/2} (p_i-p_{i-1}), \quad c = 1, \ldots,C.
\end{equation}

In [ ]:
def compute_residual(vars0, vars):
    p0 = vars0[::nc]
    p = vars[::nc]
    p_m = p[:-1]
    p_p = p[1:]
    
    """ Calculate the props of the previous time step """
    # Delete the first variable (pressure) from each cell in vars0 and extract the rest (composition)
    z0 = np.delete(vars0, np.arange(0, len(vars0), nc))
    reshaped_array0 = z0.reshape(nx, nc - 1)
    # Get the overall mole fractions of all the components
    zc0_full = np.hstack([reshaped_array0, 1 - np.sum(reshaped_array0, axis=1, keepdims=True)]).flatten()

    # Pre-allocate total density (for accumulation term)
    rho_t0 = np.zeros(nx)
    for i in range(nx):
        x, rho, s = compute_props(p0[i], zc0_full[i * nc:(i + 1) * nc])        
        rho_t0[i] = s[0] * rho[0] + s[1] * rho[1]
        
    """ Calculate the props of the current time step """
    # Delete the first variable (pressure) from each cell in vars and extract the rest (composition)
    z = np.delete(vars, np.arange(0, len(vars), nc))
    reshaped_array = z.reshape(nx, nc - 1)
    # Get the overall mole fractions of all the components
    zc_full = np.hstack([reshaped_array, 1 - np.sum(reshaped_array, axis=1, keepdims=True)]).flatten()

    # Pre-allocate props for both accumulation and upstream
    sG = np.zeros(nx)
    rhoG = np.zeros(nx)
    rhoL = np.zeros(nx)
    xG = np.zeros((nx, nc))
    xL = np.zeros((nx, nc))
    iup = np.arange(nx - 1)
    
    for i in range(nx):

        x, rho, s = compute_props(p[i], zc_full[i * nc:(i + 1) * nc])

        sG[i] = s[0]
        xG[i, :], xL[i, :] = x[0, :], x[1, :]
        rhoG[i], rhoL[i] = rho

        if i < nx - 1: # arrange upwind array
            if p[i] > p[i + 1]: 
                iup[i] = i
            else:
                iup[i] = i + 1

    """ Construct residual equations """
    residual = np.zeros(nx * nc)
    rho_t = sG * rhoG + (1 - sG) * rhoL
    for c in range(nc):  # nc mass conservation equations
        residual[c::nc] = PV * ((1 + compr_rock * (p - p_ref)) * rho_t * zc_full[c::nc] - 
                                (1 + compr_rock * (p0 - p_ref)) * rho_t0 * zc0_full[c::nc]) / dt

        beta = rhoG[iup] * xG[iup, c] * sG[iup] ** nG / miuG + rhoL[iup] * xL[iup, c] * (1 - sG[iup]) ** nL / miuL

        flux_c = - geometric_trans * beta * (p_p - p_m)

        for j in range(nx - 1):
            residual[j * nc + c] += flux_c[j]
            residual[(j + 1) * nc + c] -= flux_c[j]

    
    return residual

### Constructing the Jacobian matrix using numerical differentiation

Here we use numerical derivatives to find Jacobian. When residual is implemented as a direct function of your nonlinear unknowns, it is very easy to make:

\begin{eqnarray}
\frac{\partial r_i(p,z)}{\partial p_j} &=& \frac{r_i(p+\delta_{ij}\varepsilon, z) - r_i(p,z)}{\varepsilon},\\
\frac{\partial r_i(p,z)}{\partial z_j} &=& \frac{r_i(p, z+\delta_{ij}\varepsilon) - r_i(p,z)}{\varepsilon}.
\end{eqnarray}

{\it Note: make sure that the perturbation parameter $\varepsilon$  stays small, between $10^{-4}-10^{-8}$; make sure that it is adjusted to the characteristic value of your unknowns, e.g. $p\approx 10^5-10^7$ Pa or $10^1-10^3$ bars, while $z\in[0,1]$.}

In [ ]:
def construct_Jacobian_matrix(vars0, vars):
    jac = np.zeros((nx * nc, nx * nc))
    residual = compute_residual(vars0, vars)

    for i in range(nx):  # cell i
        vars[i * nc + 0] += eps_p
        jac[:, i * nc + 0] = (compute_residual(vars0, vars) - residual) / eps_p
        vars[i * nc + 0] -= eps_p
        for j in range(1, nc):
            vars[i * nc + j] += eps_z
            jac[:, i * nc + j] = (compute_residual(vars0, vars) - residual) / eps_z
            vars[i * nc + j] -= eps_z

    return residual, jac

### Main simulator function

In [ ]:
def simulate(vars0):
    vars = np.array(vars0, copy=True)

    total_iter_counter = 0  # counter for total iterations
    total_runtime = 0

    start = time.time()
    
    for ts_counter in range(num_ts):
        vars0 = np.copy(vars)

        ts_runtime = 0
        
        for iter_counter in range(max_num_NR_iterations):
            residuals, jac = construct_Jacobian_matrix(vars0, vars)

            res_norm = np.linalg.norm(residuals)

            if res_norm < NR_tolerance:
                total_iter_counter += iter_counter + 1
                print("TS = %d, iter = %d, res = %e" % (ts_counter, iter_counter, res_norm))
                break

            elif iter_counter == max_num_NR_iterations - 1 and res_norm > NR_tolerance:
                raise Exception("Solution did not converge!")

            update = np.linalg.solve(jac, -residuals)
            vars += update  # vars are updated here by adding the update


    print('Total runtime = ', time.time() - start)
    return vars

### Specify the properties of the model

In [ ]:
## Define domain geometry
nx = 50  # number of reservoir grid cells
dx, dy, dz = 1, 5, 5   # meters

## Specify rock props
poro = 0.2
perm = 1   # milli Darcy
compr_rock = 0

## Calculate the geometric part of transmissibility
Darcy_coeff = 0.008526   # Darcy coefficient for unit conversion
A = dy * dz
geometric_trans = Darcy_coeff * perm * A / dx

## Pore volume
PV = np.array([dx * dy * dz] * nx) * poro 

### Define fluid properies

In [ ]:
nc = 2  # number of components
max_num_phases = 2 # numebr of phases
# components: CO2, H2O
K_values = np.array([10, 0.2])  # equilibrium ratios

# Define pressure-dependent phase densities
p_ref = 1  # bar
rho_ref_gas = 200  # kg/m3
rho_ref_liquid = 600
compr_gas = 1e-3  # 1/bar
compr_liquid = 1e-5  # 1/bar

# Use constant phase viscosities
miuL = 0.5
miuG = 0.05

# Use the Brooks-Corey model for phase relative permeabilities
nG = 2
nL = 2

### Set initial and boundary conditions

In [ ]:
# initial conditions
z_init = [0.01, 0.99] # initial fluid composition
cell_init = [150] + z_init[:-1] # pressure and nc-1 independent fractions
vars0 = np.array(cell_init * nx)

# boudnary conditions
z_inj = [0.99, 0.01]  # injected fluid composition
vars0[0:nc] = [250] + z_inj[:-1]  # injection pressure and nc-1 independent fractions

# boundary volumes (large to preserve bc) 
PV[0]  = 1e8
PV[-1] = 1e8

### Simulation parameters

In [ ]:
dt = 5  # day
num_ts = 10  # number of time steps

max_num_NR_iterations = 200
NR_tolerance = 1e-3

# peretrubations for preseure and fractions
eps_p = 0.01 
eps_z = 0.0001

### Start simulation

In [ ]:
vars = simulate(vars0)

### Visualize results

In [ ]:
fig = plt.figure()   
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
x = np.arange(dx / 2, nx * dx, dx)

ax = axes[0]
ax.grid()
ax.plot(x, vars[0::nc])
ax.set_xlabel('x [meter]')
ax.set_ylabel("Pressure [bar]")

ax = axes[1]
ax.grid()

for c in range(1, nc):
    ax.plot(x, vars[c::nc])
ax.set_xlabel('x [meter]')
ax.set_ylabel("Overall fraction [-]")

plt.show()

### <font color='blue'>Assignment: change simulator for 3 component system (add methane)